In [14]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`
# (2) save links to ~/Downloads/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run make importdb`
links = pd.read_csv('data/export.csv')
links

/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83772,140980,https://www.cold-takes.com/investigating-music...,Investigating musical genius by listening to t...,\N,\N,2022-02-03 19:26:47.232902+00,2022-02-03 19:26:47.232902+00,NaN,\N,Holden,3.0,0.0,\N
83773,140981,https://fivethirtyeight.com/features/how-a-cou...,How A Court Ruling In Alabama Could Boost Blac...,\N,\N,2022-02-03 19:26:47.234343+00,2022-02-03 19:26:47.234343+00,NaN,\N,538,9.0,0.0,\N
83774,140982,https://www.askamanager.org/2022/02/how-can-i-...,how can I get time off work for therapy?,\N,\N,2022-02-03 19:26:47.235822+00,2022-02-03 19:26:47.235822+00,NaN,\N,AskManager,6.0,0.0,\N
83775,140983,https://www.centreforeffectivealtruism.org/blo...,CEA’s 2017 review and 2018 plans,\N,\N,2022-02-03 19:26:47.237185+00,2022-02-03 19:26:47.237185+00,NaN,\N,CEA,19.0,0.0,\N


In [15]:
links['added'].max()

'2022-02-03 19:26:47.238632+00'

In [16]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


1299       1306
1300       1307
1301       1308
1302       1309
1303       1310
          ...  
83761    140968
83762    140969
83763    140970
83764    140971
83765    140972
Name: id, Length: 6588, dtype: int64

In [17]:
link_file = open('/Users/peterhurford/Downloads/links.txt', 'r')
new_links = link_file.readlines()

In [18]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = links.append(entry, ignore_index=True)

links

...1/34
...11/34
...21/34
...31/34
...34/34


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83805,141013,https://twitter.com/andy_matuschak/status/1489...,"Andy Matuschak on Twitter: ""In the US deaths a...",\N,\N,2022-02-03 17:05:12.138677,2022-02-03 17:05:12.138692,\N,\N,Custom,91.0,0.0,NaN
83806,141014,https://twitter.com/michael_nielsen/status/148...,"Michael Nielsen on Twitter: ""A trick I learned...",\N,\N,2022-02-03 17:05:12.360807,2022-02-03 17:05:12.360822,\N,\N,Custom,68.0,0.0,NaN
83807,141015,https://twitter.com/ben_j_todd/status/14892939...,"Benjamin Todd on Twitter: ""1) People give very...",\N,\N,2022-02-03 17:05:12.535095,2022-02-03 17:05:12.535113,\N,\N,Custom,11.0,0.0,NaN
83808,141016,https://twitter.com/robertwiblin/status/148934...,"Robert Wiblin on Twitter: ""How did I somehow n...",\N,\N,2022-02-03 17:05:12.618517,2022-02-03 17:05:12.618534,\N,\N,Custom,60.0,0.0,NaN


In [19]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,productivity,Custom,10.0,0.0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,productivity,Custom,9.0,0.0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,effectivealtruism,Custom,85.0,0.0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1,politicalscience,Custom,82.0,0.0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,entrepreneurship,Custom,59.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83805,141013,https://twitter.com/andy_matuschak/status/1489...,"Andy Matuschak on Twitter: ""In the US deaths a...",\N,\N,2022-02-03 17:05:12.138677,2022-02-03 17:05:12.138692,\N,\N,Custom,91.0,0.0,NaN
83806,141014,https://twitter.com/michael_nielsen/status/148...,"Michael Nielsen on Twitter: ""A trick I learned...",\N,\N,2022-02-03 17:05:12.360807,2022-02-03 17:05:12.360822,\N,\N,Custom,68.0,0.0,NaN
83807,141015,https://twitter.com/ben_j_todd/status/14892939...,"Benjamin Todd on Twitter: ""1) People give very...",\N,\N,2022-02-03 17:05:12.535095,2022-02-03 17:05:12.535113,\N,\N,Custom,11.0,0.0,NaN
83808,141016,https://twitter.com/robertwiblin/status/148934...,"Robert Wiblin on Twitter: ""How did I somehow n...",\N,\N,2022-02-03 17:05:12.618517,2022-02-03 17:05:12.618534,\N,\N,Custom,60.0,0.0,NaN


In [20]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83805,141013,https://twitter.com/andy_matuschak/status/1489...,"Andy Matuschak on Twitter: ""In the US deaths a...",\N,\N,2022-02-03 17:05:12.138677,2022-02-03 17:05:12.138692,\N,\N,Custom,91,0,NaN
83806,141014,https://twitter.com/michael_nielsen/status/148...,"Michael Nielsen on Twitter: ""A trick I learned...",\N,\N,2022-02-03 17:05:12.360807,2022-02-03 17:05:12.360822,\N,\N,Custom,68,0,NaN
83807,141015,https://twitter.com/ben_j_todd/status/14892939...,"Benjamin Todd on Twitter: ""1) People give very...",\N,\N,2022-02-03 17:05:12.535095,2022-02-03 17:05:12.535113,\N,\N,Custom,11,0,NaN
83808,141016,https://twitter.com/robertwiblin/status/148934...,"Robert Wiblin on Twitter: ""How did I somehow n...",\N,\N,2022-02-03 17:05:12.618517,2022-02-03 17:05:12.618534,\N,\N,Custom,60,0,NaN


In [21]:
links.to_csv('data/export.csv', index=False)